Text Tutorial

https://docs.fast.ai/tutorial.text.html

In [1]:
# installing/upgrading fastai2
!pip install --upgrade fastai

from fastai.vision.all import *


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.2 MB/s eta 0:00:00


In [2]:
%matplotlib inline


In [3]:
from fastai.text.all import *

In [ ]:
path = untar_data(URLs.IMDB)
path.ls()

(#7) [Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/tmp_clas')]

In [ ]:
(path/'train').ls()

(#4) [Path('/root/.fastai/data/imdb/train/neg'),Path('/root/.fastai/data/imdb/train/unsupBow.feat'),Path('/root/.fastai/data/imdb/train/labeledBow.feat'),Path('/root/.fastai/data/imdb/train/pos')]

In [ ]:
dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')

In [ ]:
dls.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj warning : xxmaj does contain spoilers . \n\n xxmaj open xxmaj your xxmaj eyes \n\n xxmaj if you have not seen this film and plan on doing so , just stop reading here and take my word for it . xxmaj you have to see this film . i have seen it four times so far and i still have n't made up my mind as to what exactly happened in the film . xxmaj that is all i am going to say because if you have not seen this film , then stop reading right now . \n\n xxmaj if you are still reading then i am going to pose some questions to you and maybe if anyone has any answers you can email me and let me know what you think . \n\n i remember my xxmaj grade 11 xxmaj english teacher quite well . xxmaj",pos
2,"xxbos i thought that xxup rotj was clearly the best out of the three xxmaj star xxmaj wars movies . i find it surprising that xxup rotj is considered the weakest installment in the xxmaj trilogy by many who have voted . xxmaj to me it seemed like xxup rotj was the best because it had the most profound plot , the most suspense , surprises , most xxunk the ending ) and definitely the most episodic movie . i personally like the xxmaj empire xxmaj strikes xxmaj back a lot also but i think it is slightly less good than than xxup rotj since it was slower - moving , was not as episodic , and i just did not feel as much suspense or emotion as i did with the third movie . \n\n xxmaj it also seems like to me that after reading these surprising reviews that",pos
3,"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the sweetest and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with the oddest possible scenarios to get the two protagonists together in the end . xxmaj in fact , all its charm is innate , contained within the characters and the setting and the plot … which is highly believable to boot . xxmaj it 's easy to think that such a love story , as beautiful as any other ever told , * could * happen to you … a feeling you do n't often get from other romantic comedies",pos
4,"xxbos xxmaj in xxup nyc , seaman xxmaj michael o'hara ( orson xxmaj welles ) rescues xxmaj elsa xxmaj bannister ( rita xxmaj hayworth ) from a mugging & rape as she takes a horse & carriage through xxmaj central xxmaj park -and lives to regret it . xxmaj xxunk - haired xxmaj hayworth 's a platinum blonde in this one ; as dazzling as fresh - fallen snow -but nowhere near as pure … \n\n xxmaj to reveal any more of the convoluted plot in this seminal "" noir "" would be criminal . xxmaj it 's as deceptive as the mirrors used to cataclysmic effect in the final scenes -but the film holds far darker secrets : xxmaj from the xxup ny xxmaj times : "" childhood xxmaj shadows : xxmaj the xxmaj hidden xxmaj story xxmaj of xxmaj the xxmaj black xxmaj dahlia xxmaj murder "" by",pos
5,"xxbos xxunk ) is the developing world 's answer to xxmaj silence of the xxmaj lambs . xxmaj where ` silence ' terrorized our peace of mind , ` citizen ' exhausts and sa

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.464301,0.403392,0.816440,03:16


epoch,train_loss,valid_loss,accuracy,time
0,0.295514,0.297310,0.878800,06:59
1,0.246242,0.213751,0.912840,06:59
2,0.189195,0.202908,0.918360,06:59
3,0.146368,0.192502,0.927040,06:59


In [ ]:
learn.show_results()

,text,category,category_
0,"xxbos xxmaj there 's a sign on xxmaj the xxmaj lost xxmaj highway that says : \n\n * major xxup spoilers xxup ahead * \n\n ( but you already knew that , did n't you ? ) \n\n xxmaj since there 's a great deal of people that apparently did not get the point of this movie , xxmaj i 'd like to contribute my interpretation of why the plot makes perfect sense . xxmaj as others have pointed out , one single viewing of this movie is not sufficient . xxmaj if you have the xxup dvd of xxup md , you can "" cheat "" by looking at xxmaj david xxmaj lynch 's "" top 10 xxmaj hints to xxmaj unlocking xxup md "" ( but only upon second or third viewing , please . ) ;) \n\n xxmaj first of all , xxmaj mulholland xxmaj drive is",pos,pos
1,"xxbos "" the xxmaj moon xxmaj is xxmaj blue "" director xxmaj otto xxmaj preminger tackled even more taboo subject matter in his controversial 1955 release "" the xxmaj man with the xxmaj golden xxmaj arm . "" xxmaj whereas he had incensed the xxmaj motion xxmaj picture xxmaj association of xxmaj america with his use of the words "" virgin "" and "" mistress "" in his mild 1953 comedy "" the xxmaj moon xxmaj is xxmaj blue , "" xxmaj preminger went far beyond what any movie had attempted with "" the xxmaj man with the xxmaj golden xxmaj arm "" since xxmaj dick xxmaj powell made his law and order epic "" to the xxmaj ends of the xxmaj xxunk ) about thwarting the international traffic in narcotics . xxmaj based on xxmaj nelson xxmaj algren 's novel that won the 1950 xxmaj national xxmaj book xxmaj",pos,pos
2,"xxbos xxmaj it has been widely agreed that xxmaj hayao xxmaj miyazaki is a master at his craft when it comes to combining rich animation with thoughtful story lines and similarly imaginative characters . xxmaj his movies , from xxup nausicaa , xxup totoro , xxup kiki , xxup laputa , and xxup mononoke to the recent xxup howl 's xxup moving xxup castle are all not only gorgeously rendered in terms of art , but in terms of movie - making as well . xxmaj can this man do no wrong ? xxmaj not really , but it is impossible to expect everyone of his movies to always be five star marvels . xxmaj his newest film , xxup ponyo , an unashamedly family - friendly tale of a "" goldfish out of water "" , is as lushly animated and alive with interesting characters as you would expect",pos,pos
3,"xxbos xxmaj my qualifications for this review ? i own all the xxmaj alien and xxmaj predator movies & i have and have read almost all the books i can find that are related to this series . \n\n i can safely say , this movie is a xxmaj stinker . xxmaj save your money & do n't waste your time . xxmaj if you like mindless action , mindless gore , no plot to speak of & like being taken by xxmaj hollywood , see the movie . \n\n xxmaj if you are a serious xxmaj alien series fan , send a message to the over stuffed , over paid suits in xxmaj hollywood & 20th xxmaj century xxmaj fox & do n't give them a penny . \n\n xxmaj this movie has so many plot holes in it you could sieve pasta through it . xxmaj read the",neg,neg
4,"xxbos xxmaj on account of my unfortunately not being able to find them anywhere , i have not gotten to try any of the other entries in the series , although i certainly would not mind , and trust me , i have looked . xxmaj for anyone who does not know , this is a point - and - click adventure title . xxmaj that means that the mouse is what you use to interface with everything that you can do so with in this , though there is one particular case in this where that is inaccurate . i wo n't spoil it here , for anyone who have n't yet tried it . xxmaj nevertheless , regardless of how little experience you have with computers , you can sit right down and try this . xxmaj there is n't even terribly many bits of this where you",pos,pos
5,"xxbos xxmaj dracula 3 xxrep 3 0 or xxmaj van xxmaj helsing "" dracula 's xxmaj revenge "" ( cheap cash in on another lame xxmaj vampire flick ) as i saw it is a master c

In [ ]:
learn.predict("I really liked that movie!")

('pos', tensor(1), tensor([0.0012, 0.9988]))

In [4]:
path = untar_data(URLs.IMDB)
path.ls()

(#7) [Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/tmp_clas')]

In [ ]:
(path/'train').ls()

(#4) [Path('/root/.fastai/data/imdb/train/neg'),Path('/root/.fastai/data/imdb/train/unsupBow.feat'),Path('/root/.fastai/data/imdb/train/labeledBow.feat'),Path('/root/.fastai/data/imdb/train/pos')]

In [ ]:
dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')

In [ ]:
dls.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos "" fever xxmaj pitch "" is n't a bad film ; it 's a terrible film . \n\n xxmaj is it possible xxmaj american movie audiences and critics are so numbed and lobotomized by the excrement that xxmaj hollywood churns out that they 'll praise to the skies even a mediocre film with barely any laughs ? xxmaj that 's the only reason i can think of why this horrible romantic comedy ( and i use that term loosely because there 's nothing funny in this film ) is getting good reviews . \n\n i sat through this film stunned that screenwriters xxmaj lowell xxmaj ganz and xxmaj babaloo xxmaj mandel would even for an instant think their script was funny . \n\n xxmaj the brilliant xxmaj nick xxmaj hornby usually translates well to film . xxmaj he adapted "" fever xxmaj pitch "" for a xxmaj british film",neg
2,"xxbos xxmaj well , on it 's credit side ( if it can be said to have one ) , xxmaj timothy xxmaj hines xxup did manage to capture the original setting of xxup h.g . xxmaj wells ' outstanding novella . xxmaj but other than that - well , to call a spade a spade - it sucks bigtime . xxmaj what the xxmaj master xxmaj ed xxmaj wood could have done with the alleged $ 20 million dollar budget ! xxmaj timothy xxmaj hines really does make xxmaj mr . xxmaj wood , who was a flawed genius anyway , look like the best filmmaker of all time . xxmaj the special effects ( i guess you 'd call them that ) are not even up to computer game standards . xxmaj the acting is , well , perhaps about dinner theater comparable , and the accents are",neg
3,"xxbos xxmaj on xxmaj sunday xxmaj july 27 , 1997 , the first episode of a new science fiction series called "" stargate xxup sg-1 "" was broadcast on xxmaj showtime . a spin - off of and sequel to the 1994 film "" stargate "" starring xxmaj kurt xxmaj russell and xxmaj james xxmaj spader , the series begins approximately one year after the events portrayed in the film . xxmaj for ten seasons , it chronicled the adventures and misadventures of an intrepid team of explorers known as xxup sg-1 . xxmaj originally , the series starred xxmaj richard xxmaj dean xxmaj anderson as xxmaj colonel xxmaj jack o'neill ( two "" xxunk ! ) , xxmaj michael xxmaj shanks as xxmaj dr . xxmaj daniel xxmaj jackson , xxmaj amanda xxmaj tapping as xxmaj captain xxmaj samantha xxmaj carter , xxmaj christopher xxmaj judge as xxmaj teal'c",pos
4,"xxbos xxrep 6 * xxup warning : xxup may xxup contain xxup spoilers xxrep 14 * \n\n xxmaj so who are these "" mystery xxmaj men ? "" xxmaj simply put , the xxmaj mystery xxmaj men are a group of sub - heroes desperately trying to live out their adolescent fantasy lives while botching both their real identities and their super identities . xxmaj the xxmaj shoveller ( bill xxmaj macy ) works construction during the day , and at night , leaves his wife and kids at home while he cruises the street looking for crimes to tackle with his extraordinary and unique xxmaj shovel - fighting style . xxmaj the xxmaj blue xxmaj raja ( hank xxmaj azaria ) sells silverware to newlyweds by day and flings xxunk at crackpot villians by night , if his mom is n't keeping him busy with the latest s

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn.fine_tune(4, 1e-2)

In [ ]:
imdb = DataBlock(blocks=(TextBlock.from_folder(path), CategoryBlock),
                 get_items=get_text_files,
                 get_y=parent_label,
                 splitter=GrandparentSplitter(valid_name='test'))

In [ ]:
dls = imdb.dataloaders(path)

##ULMFiT##

ULMFiT (Universal Language Model Fine-tuning) is a method for improving the performance of text models on specific tasks by leveraging pre-trained models and transfer learning. Here's the gist of the method:

Pre-training: ULMFiT starts with a pre-trained language model on a massive dataset like Wikipedia text. This model learns general language understanding.

Gradual Fine-tuning:  Then, ULMFiT fine-tunes the pre-trained model for the target task in stages:

Fine-tune the Language Model: The model is further trained on the target task data, but not all parts are adjusted equally.

Gradual Unfreezing: Layers closer to the output (more specific knowledge) are adjusted more significantly, while earlier layers (general understanding) are adjusted less to prevent forgetting their core function.

Fine-tune the Classifier: Finally, a classifier specific to the target task (e.g., sentiment analysis) is added on top of the fine-tuned language model and trained.

ULMFiT's strength lies in its ability to achieve good results even with limited labeled data for the target task, because the model benefits from the pre-trained knowledge.

In [5]:
dls_lm = TextDataLoaders.from_folder(path, is_lm=True, valid_pct=0.1)

In [6]:
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos xxmaj did anyone else notice that xxmaj patrick xxmaj stewart was not actually on location in xxmaj thailand with the rest of the cast . xxmaj all his scenes were in a studio or against a ' blue ' screen , even when appearing to be in scenes with other actors - probably the best xxmaj special xxmaj effect in this film . xxmaj perhaps he did not realise how bad","xxmaj did anyone else notice that xxmaj patrick xxmaj stewart was not actually on location in xxmaj thailand with the rest of the cast . xxmaj all his scenes were in a studio or against a ' blue ' screen , even when appearing to be in scenes with other actors - probably the best xxmaj special xxmaj effect in this film . xxmaj perhaps he did not realise how bad the"
1,"for xxmaj gangsters with xxmaj xxunk xxmaj ambitions . \n\n xxmaj snipes delivers in the action xxunk is in half - blade mode xxunk actually does well in the first 15 minutes of the xxmaj detonator , before descending into pure sleepwalking mode that ruined the xxunk does more than look xxunk is some conviction to her xxunk lot of the actors look familiar , and have been in the last 3","xxmaj gangsters with xxmaj xxunk xxmaj ambitions . \n\n xxmaj snipes delivers in the action xxunk is in half - blade mode xxunk actually does well in the first 15 minutes of the xxmaj detonator , before descending into pure sleepwalking mode that ruined the xxunk does more than look xxunk is some conviction to her xxunk lot of the actors look familiar , and have been in the last 3 or"
2,"emotional pain , trying our best and sometimes not meeting the expectations of others , and the love and compassionate that can bring us together -- its all there in this powerfully moving film . \n\n xxmaj the story begins in xxmaj ireland , with a young wife and mother , who learns that her younger brother , xxmaj declan , ( also an adult ) , is in a hospital ,","pain , trying our best and sometimes not meeting the expectations of others , and the love and compassionate that can bring us together -- its all there in this powerfully moving film . \n\n xxmaj the story begins in xxmaj ireland , with a young wife and mother , who learns that her younger brother , xxmaj declan , ( also an adult ) , is in a hospital , that"
3,"but i just abhor when i can figure out what is going to happen next . xxmaj also , i hated to see good actors like xxmaj lena xxmaj olin and xxmaj giancarlo xxmaj giannini sweat so much over this piece of dreck . xxmaj finally , xxmaj anna xxmaj paquin as the star of a horror film ? xxmaj this must be a joke . xxmaj she is cute but she","i just abhor when i can figure out what is going to happen next . xxmaj also , i hated to see good actors like xxmaj lena xxmaj olin and xxmaj giancarlo xxmaj giannini sweat so much over this piece of dreck . xxmaj finally , xxmaj anna xxmaj paquin as the star of a horror film ? xxmaj this must be a joke . xxmaj she is cute but she is"
4,"as good as the first in my opinion , ca n't wait for the third one to come out ! xxbos xxmaj upon completing this infernal piece of trash , a friend and i swore a solemn vow never to again speak of how we had just trashed away the last 90 minutes of our lives . xxmaj this film is completely pointless , a two dimensional hero and heroin who we","good as the first in my opinion , ca n't wait for the third one to come out ! xxbos xxmaj upon completing this infernal piece of trash , a friend and i swore a solemn vow never to again speak of how we had just trashed away the last 90 minutes of our lives . xxmaj this film is completely pointless , a two dimensional hero and heroin who we ca"


In [7]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

Switched from Kaggle to Colab as Kaggle was unable to run these simple examples without timing out, and was also having problems saving the notebooks to github.

In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time


In [ ]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.067464,3.918259,0.304209,50.312771,26:08
1,3.966590,3.827632,0.314044,45.953579,26:07
2,3.898737,3.782675,0.319294,43.933418,26:02


In [ ]:
learn.save_encoder('finetuned')

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
         for _ in range(N_SENTENCES)]